In [1]:
import sys
sys.path.append('../../')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
from data.cifar10 import load_data
BATCH_SIZE = 50
train_iter, test_iter = load_data(batch_size=BATCH_SIZE, resize=(96, 96))

In [2]:
class Inception(tf.keras.Model):
    
    def __init__(self, c1, c2, c3, c4):
        super(Inception, self).__init__()
        self.p1_1 = tf.keras.layers.Conv2D(c1, 1, activation='relu')
        self.p2_1 = tf.keras.layers.Conv2D(c2[0], 1, activation='relu')
        self.p2_2 = tf.keras.layers.Conv2D(c2[1], 3, activation='relu', padding='same')
        self.p3_1 = tf.keras.layers.Conv2D(c3[0], 1, activation='relu')
        self.p3_2 = tf.keras.layers.Conv2D(c3[1], 5, activation='relu', padding='same')
        self.p4_1 = tf.keras.layers.MaxPool2D(3, 1, padding='same')
        self.p4_2 = tf.keras.layers.Conv2D(c4, 1, activation='relu')
    
    def call(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return tf.keras.layers.Concatenate()([p1, p2, p3, p4])

In [3]:
def b1():
    return tf.keras.Sequential([
        # filter=64, kernel_size=7  strides=2
        tf.keras.layers.Conv2D(64, 7, strides=2, padding='same', activation='relu'),
        # kernel_size=3, strides=2
        tf.keras.layers.MaxPool2D(3, 2, padding='same')
    ])

def b2():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, 1, activation='relu'),
        tf.keras.layers.Conv2D(192, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(3, 2, padding='same')
    ])

def b3():
    return tf.keras.Sequential([
        Inception(64, (96, 128), (16, 32), 32),
        Inception(128, (128, 192), (32, 96), 64),
        tf.keras.layers.MaxPool2D(3, 2, padding='same')
    ])

def b4():
    return tf.keras.Sequential([
        Inception(192, (96, 208), (16, 48), 64),
        Inception(160, (112, 224), (24, 64), 64),
        Inception(128, (128, 256), (24, 64), 64),
        Inception(112, (144, 288), (32, 64), 64),
        Inception(256, (160, 320), (32, 128), 128),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')
    ])

def b5():
    return tf.keras.Sequential([
        Inception(256, (160, 320), (32, 128), 128),
        Inception(384, (192, 384), (48, 128), 128),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Flatten()
    ])

In [4]:
net = tf.keras.Sequential([tf.keras.layers.Input(shape=(96, 96, 3)), b1(), b2(), b3(), b4(), b5(), tf.keras.layers.Dense(10, activation='softmax')])

In [5]:
net.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3), metrics=['accuracy'])

In [6]:
net.fit(train_iter, epochs=10)

Epoch 1/10
1000/1000 [==============================] - 28s 20ms/step - loss: 2.2388 - accuracy: 0.1547
Epoch 2/10
1000/1000 [==============================] - 21s 21ms/step - loss: 1.7116 - accuracy: 0.3712
Epoch 3/10
1000/1000 [==============================] - 21s 21ms/step - loss: 1.4182 - accuracy: 0.4879
Epoch 4/10
1000/1000 [==============================] - 21s 21ms/step - loss: 1.2388 - accuracy: 0.5618
Epoch 5/10
1000/1000 [==============================] - 21s 21ms/step - loss: 1.1245 - accuracy: 0.6081
Epoch 6/10
1000/1000 [==============================] - 21s 21ms/step - loss: 1.0502 - accuracy: 0.6406
Epoch 7/10
1000/1000 [==============================] - 21s 21ms/step - loss: 1.0003 - accuracy: 0.6601
Epoch 8/10
1000/1000 [==============================] - 21s 21ms/step - loss: 0.9587 - accuracy: 0.6760
Epoch 9/10
1000/1000 [==============================] - 21s 21ms/step - loss: 0.9561 - accuracy: 0.6930
Epoch 10/10
1000/1000 [==============================] - 21s 21m